In [ ]:
import re
import speech_recognition as sr
from gtts import gTTS
import pyttsx3
import IPython.display as ipd
import os
from PIL import Image
import pytesseract
import tkinter as tk
from tkinter import filedialog
import threading

# Set your Tesseract path here if needed (Windows example)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def multilingual_math_solver(question, lang='en'):
    """
    Solve simple arithmetic expressions with localized keywords.
    Supports addition, subtraction, multiplication, division.
    Handles integers and decimals.
    """
    q = question.lower().strip()

    localized_ops = {
        'en': {'add': ['+', 'plus', 'add', 'sum', 'and'], 
               'sub': ['-', 'minus', 'subtract'], 
               'mul': ['*', 'x', 'times', 'multiplied by'], 
               'div': ['/', '÷', 'divided by']},
        'am': {'add': ['ሲደመር', 'አክል'], 
               'sub': ['አነስ'], 
               'mul': ['ተባዛ', 'እጣ'], 
               'div': ['አካፋፍ']},
        'om': {'add': ['walitti ida', 'iduu'], 
               'sub': ['hir’isa', 'hirisu'], 
               'mul': ['baasi', 'baasuu'], 
               'div': ['qoodami', 'qoodu']},
        'ti': {'add': ['ኣኽልኩም', 'ደሓን ወሲኑ'], 
               'sub': ['ኣቐናጸፉ'], 
               'mul': ['ተደመሰሰ'], 
               'div': ['ተኸፋፈለ']}
    }

    ops = localized_ops.get(lang, localized_ops['en'])

    # Try localized keywords first with decimal numbers
    for op_key, keywords in ops.items():
        for keyword in keywords:
            # Regex to match decimal numbers around the operator keyword
            pattern = rf'(\d+(\.\d+)?)\s*{re.escape(keyword)}\s*(\d+(\.\d+)?)'
            match = re.search(pattern, q)
            if match:
                a, _, b, _ = match.groups()
                a, b = float(a), float(b)
                result = {
                    'add': a + b,
                    'sub': a - b,
                    'mul': a * b,
                    'div': round(a / b, 2) if b != 0 else 'undefined'
                }.get(op_key, 'unknown')

                templates = {
                    'en': f"{a} {keyword} {b} equals {result}.",
                    'am': f"{a} {keyword} {b} እኩል {result} ነው።",
                    'om': f"{a} {keyword} {b} = {result}.",
                    'ti': f"{a} {keyword} {b} ተኸፊሉ {result} እዩ።"
                }
                return templates.get(lang, templates['en'])

    # If no localized keyword matched, try math symbols and words
    pattern = r'(\d+(\.\d+)?)\s*(\+|\-|\*|x|times|÷|/|divided by|multiplied by|plus|minus|add|subtract)\s*(\d+(\.\d+)?)'
    match = re.search(pattern, q)
    if match:
        a, _, op, b, _ = match.groups()
        a, b = float(a), float(b)

        # Normalize operation keywords
        op_map = {
            '+': 'add', 'plus': 'add', 'add': 'add', 'and': 'add',
            '-': 'sub', 'minus': 'sub', 'subtract': 'sub',
            '*': 'mul', 'x': 'mul', 'times': 'mul', 'multiplied by': 'mul',
            '/': 'div', '÷': 'div', 'divided by': 'div'
        }

        op_key = op_map.get(op, None)
        if not op_key:
            return "Unknown operation."

        result = {
            'add': a + b,
            'sub': a - b,
            'mul': a * b,
            'div': round(a / b, 2) if b != 0 else 'undefined'
        }.get(op_key, 'unknown')

        templates = {
            'en': f"{a} {op} {b} equals {result}.",
            'am': f"{a} {op} {b} እኩል {result} ነው።",
            'om': f"{a} {op} {b} = {result}.",
            'ti': f"{a} {op} {b} ተኸፊሉ {result} እዩ።"
        }
        return templates.get(lang, templates['en'])

    return {
        'en': "Sorry, I couldn't understand the math question.",
        'am': "ይቅርታ፣ የሂሳብ ጥያቄውን አላስተዋወቅሁም።",
        'om': "Dhiifama, gaaffii herregaa hin hubadhu.",
        'ti': "ይቕረታ፣ ጥያቄ ሂሳብ ኣይተረደደን።"
    }.get(lang, "Sorry, I couldn't understand the math question.")


def speak_text(text, lang='en'):
    """
    Text-to-Speech with gTTS or pyttsx3 based on language.
    Deletes previous audio file before saving new one.
    """
    try:
        audio_file = "response.mp3"
        if os.path.exists(audio_file):
            os.remove(audio_file)

        # Prefer gTTS for all supported languages if available
        supported_gtts_langs = ['en', 'am', 'om', 'ti']
        if lang in supported_gtts_langs:
            tts = gTTS(text=text, lang=lang)
            tts.save(audio_file)
            return ipd.Audio(audio_file)
        else:
            # Fallback to pyttsx3 for unsupported languages
            engine = pyttsx3.init()
            engine.say(text)
            engine.runAndWait()
            return None
    except Exception as e:
        print(f"⚠️ TTS error: {e}")
        return None


def ocr_math_image(image_path, lang='eng'):
    """
    Extract text from image using pytesseract and solve math question.
    """
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image, lang=lang)
        print("Extracted Text:", text.strip())

        # Map OCR lang to solver lang
        solver_lang_map = {'amh': 'am', 'orm': 'om', 'tir': 'ti'}
        solver_lang = solver_lang_map.get(lang, 'en')

        answer = multilingual_math_solver(text, lang=solver_lang)
        print("Answer:", answer)
        return answer
    except Exception as e:
        print(f"⚠️ OCR error: {e}")
        return "Error extracting text from image."


def voice_input(lang='en'):
    """
    Capture voice input and convert to text using Google's Speech Recognition with language specified.
    """
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        print("🎙️ Speak your math question...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        lang_map = {'en': 'en-US', 'am': 'am-ET', 'om': 'om-ET', 'ti': 'ti-ER'}
        language_code = lang_map.get(lang, 'en-US')
        question = recognizer.recognize_google(audio, language=language_code)
        print("🔎 You said:", question)
        return question
    except Exception as e:
        print(f"⚠️ Speech recognition error: {e}")
        return None


# GUI Setup

def on_solve():
    question = entry.get().strip()
    if not question:
        status_label.config(text="Please enter a math question.")
        return

    lang = lang_var.get()
    answer = multilingual_math_solver(question, lang=lang)
    answer_label.config(text=answer)
    status_label.config(text="")

    # Speak answer asynchronously to keep UI responsive
    threading.Thread(target=lambda: speak_text(answer, lang)).start()


def on_voice():
    lang = lang_var.get()
    question = voice_input(lang=lang)
    if question:
        entry.delete(0, tk.END)
        entry.insert(0, question)
        status_label.config(text="Heard: " + question)
    else:
        status_label.config(text="Could not understand audio.")


def on_upload_image():
    path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg *.bmp")])
    if path:
        lang = lang_var.get()
        # Map language for OCR
        lang_map = {'en': 'eng', 'am': 'amh', 'om': 'eng', 'ti': 'eng'}
        ocr_lang = lang_map.get(lang, 'eng')
        answer = ocr_math_image(path, lang=ocr_lang)
        answer_label.config(text=answer)
        status_label.config(text="")

        threading.Thread(target=lambda: speak_text(answer, lang)).start()


# Initialize main window
root = tk.Tk()
root.title("Multilingual Math Solver")

# Language selector variable
lang_var = tk.StringVar(value='en')

# Widgets
tk.Label(root, text="Enter math question:").pack(pady=(10, 0))

entry = tk.Entry(root, width=50)
entry.pack(padx=10, pady=5)
entry.insert(0, "e.g. 5 plus 3 or 7 - 2")

tk.Label(root, text="Select language:").pack()
tk.OptionMenu(root, lang_var, 'en', 'am', 'om', 'ti').pack(pady=5)

tk.Button(root, text="Solve", command=on_solve).pack(pady=5)
tk.Button(root, text="Speak Question", command=on_voice).pack(pady=5)
tk.Button(root, text="Upload Image for OCR", command=on_upload_image).pack(pady=5)

answer_label = tk.Label(root, text="", font=("Arial", 14), fg="blue")
answer_label.pack(pady=10)

status_label = tk.Label(root, text="", fg="red")
status_label.pack()

root.mainloop()


🎙️ Speak your math question...
🔎 You said: 2+2
